In [8]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


### PHASE 1 — DATA INGESTION

In [9]:
data = pd.read_excel("Online Retail Data Set.xlsx")


In [43]:
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [10]:
data.shape

(541909, 8)

In [12]:
data.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [45]:
data.columns = data.columns.str.lower()

In [35]:
data.columns

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country'],
      dtype='object')

### PHASE 2 — DATA STORAGE (SQL Layer)

In [12]:
load_dotenv()
db_url = os.getenv('DATABASE_URL')



In [13]:
engine = create_engine(db_url)

In [47]:
data.to_sql('online_retail_dataset',engine,if_exists='replace',index=False,schema='data_set')

909

In [48]:
data.columns

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country'],
      dtype='object')

### PHASE 3 — DATA PREPARATION (CLEANING & TRANSFORMATION)
#### 1. Quantity Validation
- Exclude or filter out all transactions with negative or zero quantity.

In [62]:
cleaned_data = data[data['quantity'] >= 1]


#### 2. Unit Price Validation
- Remove or correct all rows with UnitPrice < 0.

In [63]:
cleaned_data = cleaned_data[cleaned_data['unitprice']>= 0]

#### 3. Additional Cleaning
- Remove invoice cancellations (InvoiceNo beginning with "C").
- Drop rows where CustomerID is missing.
- Remove duplicate records.
- Create new fields:
  - Revenue = Quantity × UnitPrice
  - Month, Year, Day extracted from InvoiceDate


In [64]:
cleaned_data = cleaned_data[~cleaned_data['invoiceno'].astype(str).str.startswith('C')]


In [65]:
cleaned_data = cleaned_data.dropna(subset=['customerid'])


In [66]:
cleaned_data = cleaned_data.drop_duplicates()


In [67]:
cleaned_data['revenue'] = cleaned_data['quantity'] * cleaned_data['unitprice']

In [68]:
cleaned_data['month'] = cleaned_data['invoicedate'].dt.month

In [69]:
cleaned_data['year'] = cleaned_data['invoicedate'].dt.year

In [70]:
cleaned_data['day'] = cleaned_data['invoicedate'].dt.day

In [71]:
cleaned_data.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,revenue,month,year,day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,12,2010,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,12,2010,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,1


#### 4. Store a "cleaned" version
- Load the cleaned dataset into a new SQL table,`online_retail_clean`.

In [72]:
cleaned_data.to_sql('online_retail_clean', engine, if_exists='append',index=False, schema='data_set')

732

### PHASE 4 — DATA ANALYSIS (IN PYTHON ONLY)
Perform analytical steps to prepare insights for leadership:

#### 1. Time Series (2011 Revenue by Month)
- Filter data for Year = 2011
- Group by month and calculate revenue totals
- Identify seasonal patterns or month-to-month changes

In [88]:
revenue_2011 = cleaned_data[cleaned_data['year'] == 2011]

In [100]:
monthly_revenue = (revenue_2011.groupby('month')['revenue'].sum().reset_index())


In [101]:
monthly_revenue

,month,revenue
0,1,568101.310
1,2,446084.920
2,3,594081.760
3,4,468374.331
4,5,677355.150
5,6,660046.050
6,7,598962.901
7,8,644051.040
8,9,950690.202
9,10,1035642.450


In [106]:
monthly_revenue['MOM change'] = monthly_revenue['revenue'].diff().round(2)



In [108]:
monthly_revenue['MOM change %'] = monthly_revenue['revenue'].pct_change().round(2)* 100

In [109]:
monthly_revenue


,month,revenue,MOM change,MOM change %
0,1,568101.310,NaN,NaN
1,2,446084.920,-122016.39,-21.0
2,3,594081.760,147996.84,33.0
3,4,468374.331,-125707.43,-21.0
4,5,677355.150,208980.82,45.0
5,6,660046.050,-17309.10,-3.0
6,7,598962.901,-61083.15,-9.0
7,8,644051.040,45088.14,8.0
8,9,950690.202,306639.16,48.0
9,10,1035642.450,84952.25,9.0


#### 2. Country Performance (Excluding United Kingdom)
- Rank countries by total revenue
- Identify the top 10 revenue-generating countries
- Compute both revenue and quantity sold

In [30]:
data_not_UK = cleaned_data[cleaned_data['country'] != 'United Kingdom']

In [33]:
revenue_by_country= (data_not_UK.groupby('country')[['revenue', 'quantity']].sum().reset_index().sort_values('revenue',ascending=False))


In [34]:
revenue_by_country.head(10)

,country,revenue,quantity
23,Netherlands,285446.34,200937
10,EIRE,265262.46,140383
14,Germany,228678.40,119156
13,France,208934.31,111429
0,Australia,138453.81,84199
30,Spain,61558.56,27944
32,Switzerland,56443.95,30083
3,Belgium,41196.34,23237
31,Sweden,38367.83,36078
19,Japan,37416.37,26016


In [ ]:
revenue_by_country.tail()

,country,revenue,quantity
4,Brazil,1143.60,356
27,RSA,1002.31,352
8,Czech Republic,826.74,671
2,Bahrain,548.40,260
28,Saudi Arabia,145.92,80


In [141]:
data_not_UK.columns

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country', 'revenue', 'month', 'year',
       'day'],
      dtype='object')

#### 3. Top Customers by Revenue
- Rank all customers by total revenue
- Identify the top 10 highest-spending customers

In [26]:
revenue_by_customers = (cleaned_data.groupby('customerid')['revenue'].sum().reset_index().sort_values('revenue',ascending=False))

In [28]:
revenue_by_customers.head(10)

,customerid,revenue
1690,14646.0,280206.02
4202,18102.0,259657.30
3729,17450.0,194390.79
3009,16446.0,168472.50
1880,14911.0,143711.17
55,12415.0,124914.53
1334,14156.0,117210.08
3772,17511.0,91062.38
2703,16029.0,80850.84
0,12346.0,77183.60


In [21]:
revenue_by_customers.tail()

,customerid,revenue
4099,17956.0,12.75
3015,16454.0,6.90
1794,14792.0,6.20
3218,16738.0,3.75
685,13256.0,0.00


#### 4. Global Product Demand
- Compute total quantity sold per country
- Remove the United Kingdom
- Rank countries by demand and highlight high-opportunity markets

In [23]:
total_quantity_by_country = (cleaned_data.groupby('country')['quantity'].sum().reset_index().sort_values('quantity', ascending = False))

In [24]:
total_quantity_by_country = total_quantity_by_country[total_quantity_by_country['country'] != 'United Kingdom']

In [169]:
total_quantity_by_country

,country,quantity
23,Netherlands,200937
10,EIRE,140383
14,Germany,119156
13,France,111429
0,Australia,84199
31,Sweden,36078
32,Switzerland,30083
30,Spain,27944
19,Japan,26016
3,Belgium,23237


In [25]:
total_quantity_by_country.tail()

,country,quantity
20,Lebanon,386
4,Brazil,356
27,RSA,352
2,Bahrain,260
28,Saudi Arabia,80
